In [1]:
!pip install tokenizers
!pip install torchtext
!pip install pytorch_lightning
!pip install datasets
!pip install tensorboard

In [2]:
from config_file import get_config
from train import train_model
import torch
import torch.nn as nn
from torch import optim
from lion_pytorch import Lion


torch.cuda.amp.autocast(enabled=True)

/opt/conda/envs/pytorch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

config = get_config()
config["batch_size"] = 32
config["preload"] = False
config["num_epochs"] = 20


In [4]:
from train import get_model, get_ds, run_validation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device : {device}")

Using device : cuda


In [5]:
import warnings
from tqdm import tqdm
import os
from pathlib import Path
import torchmetrics

from torch.utils.tensorboard import SummaryWriter

Path(config["model_folder"]).mkdir(parents=True, exist_ok=True)


In [6]:
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
writer = SummaryWriter(config["experiment_name"])
loss_fn = nn.CrossEntropyLoss(ignore_index = tokenizer_src.token_to_id("[PAD]"), label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr = config["lr"], eps = 1e-9)

# optimizer = Lion(model.parameters(), lr = 1e-4/10, weight_decay=1e-2)


Max length of the source sentence : 309
Max length of the source target : 274
train_dataloader length: 910
val_dataloader length: 3234


In [7]:
MAX_LR = 10**-3
STEPS_PER_EPOCH = len(train_dataloader)
EPOCHS = config["num_epochs"]

In [8]:
scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
                                            max_lr = MAX_LR,
                                            steps_per_epoch=STEPS_PER_EPOCH,
                                            epochs=EPOCHS,
                                            div_factor=100,
                                            three_phase=False,
                                            final_div_factor=100,
                                            anneal_strategy="linear")

In [9]:
train_model(config, EPOCHS,model,train_dataloader,val_dataloader,
             optimizer,loss_fn,tokenizer_src, tokenizer_tgt,scheduler, writer)

Processing Epoch 00: 100%|██████████| 910/910 [02:03<00:00,  7.38it/s, loss=6.482, lr=[0.00017484887342004028]]
stty: standard input: Inappropriate ioctl for device


SOURCE You cannot help knowing, since God in His great mercy has revealed it to you.
TARGET Voi non potete non saperlo, quando, per Sua grande misericordia, il Signore Iddio ve lo ha rivelato.
PREDICTED — Ma non è la cosa , ma non è un ’ è , ma non è la sua .
SOURCE And what did she do? Let me hear now."
TARGET "E che cosa avete fatto? ditemelo!
PREDICTED E non è che non è la cosa ?


/opt/conda/envs/pytorch21/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/opt/conda/envs/pytorch21/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/opt/conda/envs/pytorch21/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(
Processing Epoch 01: 100%|██████████| 910/910 [02:03<00:00,  7.38it/s, loss=5.752, lr=[0.0003398790987360323]] 
stty: standard input: Inappropriate ioctl for device


SOURCE I love my child with one kind of love and her with another.'
TARGET Amo mia figlia d’un amore, lei d’un altro.
PREDICTED Mi pare che mi a me .
SOURCE He has not come yet. You say he won't forgive me, because you don't know him.
TARGET Voi dite che non perdonerà perché non lo conoscete. Nessuno lo conosce.
PREDICTED Non posso .


Processing Epoch 02: 100%|██████████| 910/910 [02:02<00:00,  7.41it/s, loss=5.037, lr=[0.0005049093240520241]] 
stty: standard input: Inappropriate ioctl for device


...

SOURCE The expression of his face and his whole figure, in uniform with crosses and white trousers trimmed with gold lace, as he went hurriedly along, reminded Levin of a hunted animal conscious that things are going badly with him.
TARGET La espressione del viso e di tutta la sua figura in divisa, con le croci e i pantaloni bianchi gallonati, mentre camminava in fretta, ricordò a Levin una bestia inseguita che vede la propria situazione farsi cattiva.
PREDICTED L ’ espressione del viso e della sua figura in tutto , che fa per tutta la corona reale e , mentre egli corre , la di Levin che gli mostrava in mostra una sciocchezza .
SOURCE I am ready to do anything to make you happy,' he went on, touched by her despair. 'What would I not do to spare you such grief as this, about I know not What!
TARGET Io sono pronto a tutto purché tu sia felice — egli diceva, commosso dalla disperazione di lei — e che cosa non farei per liberarti da qualsiasi dolore, come adesso, Anna! — diss’egli.
PREDICT

Processing Epoch 18: 100%|██████████| 910/910 [02:03<00:00,  7.38it/s, loss=1.708, lr=[7.152142857142863e-05]] 
stty: standard input: Inappropriate ioctl for device


SOURCE Wallingford, six miles above Streatley, is a very ancient town, and has been an active centre for the making of English history.
TARGET Wallingford, che è a sei miglia al di sopra di Streatley, è una città antichissima ed è stata un centro attivo nella creazione della storia inglese.
PREDICTED La nostra , che è di città a Streatley , è molto vasta e pericolosa , giacchè è stato dalla salute delle città .
SOURCE 'No – I believe it is all going to begin again,' and he listened to the prayer. 'Yes, it is coming to an end.
TARGET Hai una bella mano, ha detto. — Ed egli guardò la propria mano e la mano tozza del diacono. — Ecco, ora finirà presto — pensava. — No, pare che cominci daccapo — pensò, prestando orecchio alle preghiere. — No, finisce; ecco che si è inchinato fino a terra.
PREDICTED — No , lo credo , non credo — disse lui , ridendo e cambiò discorso . — Sì , è già una via d ’ uscita .


Processing Epoch 19: 100%|██████████| 910/910 [02:03<00:00,  7.40it/s, loss=1.720, lr=[1.0000000000005664e-07]]
stty: standard input: Inappropriate ioctl for device


SOURCE 'I say, Egor, don't you find it difficult to keep awake?'
TARGET — Be’, Egor, è difficile riuscire a non dormire?
PREDICTED — Io dico , Egor , che non ti si in questo , non è possibile ?
SOURCE "How long did you reside with him and his sisters after the cousinship was discovered?"
TARGET — Quanto tempo siete stata con le sorelle e con lui, dopo scoperta la parentela?
PREDICTED — Quanto tempo vi siete qualcuno con lui e delle sorelle lo ?
